# Part I. ETL Pipeline for Pre-Processing the Files

### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
from tabulate import tabulate
import cassandra
import os
import glob
import csv

### Create list of filepaths to process original event csv data files

In [2]:
# Check current working directory
print(f'Current working directory: {os.getcwd()}')

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(f'File path sample: {file_path_list[0]}') if file_path_list else print('File paths missing')

Current working directory: /home/workspace
File path sample: /home/workspace/event_data/2018-11-27-events.csv


### Process the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:

# Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# Extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(f'Number of data rows extracted: {len(full_data_rows_list)}')
print(f'Data row sample: {full_data_rows_list[0]}') if full_data_rows_list else print('Source data empty')

# Create a smaller event data csv file to be used to insert data into the tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Number of data rows extracted: 8056
Data row sample: ['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(f'Number of data rows in new csv file: {sum(1 for line in f)}')

Number of data rows in new csv file: 6821


### The CSV file titled <font color=red>event_datafile_new.csv</font> should now be available to work with and located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### The image below is a screenshot of how the denormalized data should appear in the <font color=red>**event_datafile_new.csv**</font> file after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Part II. Apache Cassandra Database Implementation

### Create Apache Cassandra Cluster

In [5]:
# Create cluster

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1']) # make a connection to a Cassandra instance on your local machine
    session = cluster.connect()
except Exception as exception:
    print(exception)

### Create Keyspace

In [6]:
# Create keyspace

try:
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS sparkify_db1
        WITH REPLICATION =
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
        """
    )
except Exception as exception:
    print(exception)

### Set Keyspace

In [7]:
# Set keyspace

try:
    session.set_keyspace('sparkify_db1')
except Exception as exception:
    print(exception)

### Create tables for the following queries. With Apache Cassandra, it's best practice to model the database tables on the queries that will be run.

## Queries to to be run:

### 1. Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
### Provide the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
#### NOTE: The Primary Key for this table has two fields: session_id (the partition key) and item_in_session (the clustering column). Partitioning is done by session_id and within that partition, rows are ordered by the item_in_session.

In [8]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS song_from_session_item (
    session_id       INT,
    item_in_session  INT,
    artist_name      VARCHAR,
    song_title       VARCHAR,
    song_length      DOUBLE,
    PRIMARY KEY (session_id, item_in_session)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)                    

In [9]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_from_session_item (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# QUERY 1
# Provide the name of the artist, song title, and song's length for sessionId = 338, itemInSession = 4

# Verify data

query = "SELECT artist_name, song_title, song_length FROM song_from_session_item WHERE session_id = 338 AND item_in_session = 4"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════╤═════════════════════════════════╤═══════════════╕
│ artist_name   │ song_title                      │   song_length │
╞═══════════════╪═════════════════════════════════╪═══════════════╡
│ Faithless     │ Music Matters (Mark Knight Dub) │       495.307 │
╘═══════════════╧═════════════════════════════════╧═══════════════╛


## Query 2
### Provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### NOTE: The Primary Key for this table has three fields: user_id & session_id (used together to form the composite partition key) and item_in_session (the clustering column). Since the query uses a user_id and a session_id to find the desired data, it makes sense to partition using both user_id & session_id. Within that composite partition, rows are ordered by the item_in_session.

In [11]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS song_from_user_session (
    user_id          INT,
    session_id       INT,
    item_in_session  INT,
    artist_name      VARCHAR,
    song_title       VARCHAR,
    user_first_name  VARCHAR,
    user_last_name   VARCHAR,
    PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)                    

In [12]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_from_user_session (user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# QUERY 2
# Provide the name of the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Verify data

query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM song_from_user_session WHERE user_id = 10 AND session_id = 182"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════════╤══════════════════════════════════════════════════════╤═══════════════════╤══════════════════╕
│ artist_name       │ song_title                                           │ user_first_name   │ user_last_name   │
╞═══════════════════╪══════════════════════════════════════════════════════╪═══════════════════╪══════════════════╡
│ Down To The Bone  │ Keep On Keepin' On                                   │ Sylvie            │ Cruz             │
├───────────────────┼──────────────────────────────────────────────────────┼───────────────────┼──────────────────┤
│ Three Drives      │ Greece 2000                                          │ Sylvie            │ Cruz             │
├───────────────────┼──────────────────────────────────────────────────────┼───────────────────┼──────────────────┤
│ Sebastien Tellier │ Kilometer                                            │ Sylvie            │ Cruz             │
├───────────────────┼───────────────────────────────────────────────────

## Query 3
### Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### NOTE: The Primary Key for this table has two fields: song_title (the partition key) and user_id (the clustering column). Since the query is searching for users using a particular song, it makes sense to partition using the song_title. Within that partition, rows are ordered by the user_id.

In [14]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Create table

query = ("""
CREATE TABLE IF NOT EXISTS user_from_song_listen (
    song_title       VARCHAR,
    user_id          INT,
    user_first_name  VARCHAR,
    user_last_name   VARCHAR,
    PRIMARY KEY (song_title, user_id)
    )
""")
try:
    session.execute(query)
except Exception as exception:
    print(exception)
                    

In [15]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Insert data

filepath = 'event_datafile_new.csv'

with open(filepath, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_from_song_listen (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# QUERY 3
# Provide the user name (first and last) of each person that listened to the song 'All Hands Against His Own'

# Verify data

query = "SELECT user_first_name, user_last_name FROM user_from_song_listen WHERE song_title = 'All Hands Against His Own'"
try:
    query_df = pd.DataFrame(list(session.execute(query)))
except Exception as exception:
    print(exception)

print(tabulate(query_df, headers='keys', tablefmt='fancy_grid', showindex=False))

╒═══════════════════╤══════════════════╕
│ user_first_name   │ user_last_name   │
╞═══════════════════╪══════════════════╡
│ Jacqueline        │ Lynch            │
├───────────────────┼──────────────────┤
│ Tegan             │ Levine           │
├───────────────────┼──────────────────┤
│ Sara              │ Johnson          │
╘═══════════════════╧══════════════════╛


### Drop the tables before closing out the sessions

In [17]:
## Drop the tables before closing out the sessions

drop_queries = [
    "DROP table song_from_session_item",
    "DROP table song_from_user_session",
    "DROP table user_from_song_listen"
]

for query in drop_queries:
    try:
        session.execute(query)
    except Exception as exception:
        print(exception)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()